# Initialization and Function Definitions

In [ ]:
import os
import sys

import glob

from prfi_gen.rfi_generator import NBRFI
from prfi_gen.utils import get_noised_and_normalized_array

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm.notebook import trange, tqdm

In [23]:
def create_step_function(size, position, width):
    """
    Creates an array representing a step function with steps set to 1 
    between the given position and the sum of position and width.
    
    Parameters:
    size (int): The size of the resulting step array.
    position (int): The starting index of the step in the array.
    width (int): The width of the step in the array.
    
    Returns:
    ndarray: An array representing the step function.
    """
    step_array = np.zeros(size)
    
    # Set the values within the step range to 1
    step_array[position:position+width] = 1
    
    return step_array


def get_intest(pulse, edge):
    """
    Calculates the ratio of the maximum of the on_pulse to the standard deviation of the off_pulse.
    
    Parameters:
    pulse (ndarray): One-dimensional array representing the pulse.
    edge (int): The index at which to divide the pulse into on_pulse and off_pulse.
    
    Returns:
    float: The rounded ratio of the maximum of on_pulse to the standard deviation of off_pulse.
    """
    on_pulse = pulse[:edge]
    off_pulse = pulse[edge:]
    
    return np.round(on_pulse.max() / off_pulse.std(), 1)


def get_intensity(array, loc, width):
    """
    Calculates the maximum intensity for each line in the given array.
    
    Parameters:
    array (ndarray): Two-dimensional array representing multiple lines.
    loc (int): The location index around which to calculate intensity.
    width (int): The width around the location index to consider for calculating intensity.
    
    Returns:
    float: The rounded maximum intensity calculated across all lines in the array.
    """
    ins_list = []
    for line in array:
        left_edge = loc - 3*width 
        right_edge = loc + 3*width
        shift = int(round(left_edge, 0))
        if left_edge > 0:
            pulse = np.roll(line, -1*shift)
        else:
            pulse = line
               
        edge = int(right_edge - shift)
    
        
        ins_list.append(get_intest(pulse, edge))
    
    return np.round(np.max(ins_list), 2)


def get_complex_rfi(num):
    """
    Generates complex Radio Frequency Interference (RFI) by adding multiple RFIs together.
    
    Parameters:
    num (int): The number of RFIs to generate and add together.
    
    Returns:
    ndarray: An array representing the generated complex RFI.
    """
    nbrfi = NBRFI(128, 128)
    
    rfi = nbrfi.get_rfi(np.random.uniform(0.5, 1.5), np.random.randint(1, 5), np.random.randint(10, 120))
    for _ in range(num - 1):
        rfi += nbrfi.get_rfi(np.random.uniform(0.5, 1.5), np.random.randint(1, 10), np.random.randint(10, 120))
    
    return rfi

# Setting Paths

In [3]:
PATH_MODELS = '../models/'

In [ ]:
models = glob.glob(f'{PATH_MODELS}*.keras')
models

# Visualizing Noise-Added RFIs

A series of subplots are generated to visualize the effect of added noise to Radio Frequency Interferences (RFIs), with different noise levels applied to each subplot, aiding in the examination of signal clarity and integrity under various noise conditions.

In [ ]:
nbrfi = NBRFI(128, 128)
noise_list = np.linspace(0.01, 1.5, 25)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    amp = 1
    width = 10
    loc = 60

    rfi = nbrfi.get_rfi(amp, width, loc)
    noised_rfi = get_noised_and_normalized_array(rfi, noise_list[i])
    
    chart = np.mean(noised_rfi.T, axis=0)
    chart = chart - np.median(chart)
    left_edge = loc - 3*width 
    right_edge = loc + 3*width
    shift = int(round(left_edge, 0))
    if left_edge > 0:
        pulse = np.roll(chart, -1*shift)
    else:
        pulse = chart
    edge = int(right_edge - shift)
    snr = get_intest(pulse, edge)
    plt.title(f'Ints: {snr}')
    plt.plot(chart, color='black')
    
plt.tight_layout()    
plt.show()

# Visualization of Intensity Across Different Noise Levels

The intensity of noise-added RFIs is visualized across different noise levels to observe the variation of signal intensity under different noise conditions.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    amp = 1
    width = 10
    loc = 60

    rfi = nbrfi.get_rfi(amp, width, loc)
    noised_rfi = get_noised_and_normalized_array(rfi, noise_list[i])
    ints = get_intensity(noised_rfi, loc, width)

    plt.title(f'Ints: {ints}')
    plt.imshow(noised_rfi, aspect='auto', cmap='gray')
    
plt.tight_layout()    
plt.show()

# Evaluation of Model Accuracy Across Different Noise Levels

The loaded models are evaluated under varying noise conditions to determine their accuracy and robustness in identifying signals amidst noise, and the results are documented for further analysis.

In [ ]:
num_spects = 10
labels = np.full(num_spects, 3)
noise_amp_list = np.round(np.linspace(0.01, 1.5, 50), 2)
noise_amp_list

In [ ]:
for modelname in tqdm(models, desc='Models loop'):
    name_part = os.path.basename(modelname).split('-')[0]
    loaded_model = tf.keras.models.load_model(modelname)
    intensity_array = []
    acuracy_list = []

    for n_a in tqdm(noise_amp_list, desc='SNR loop'):
        temp = []
        downsampled_array = np.empty((num_spects, 128, 128))      
        
        for i in trange(num_spects, leave=False):
            amp = 1
            width = 10
            loc = 60

            rfi = nbrfi.get_rfi(amp, width, loc)
            noised_rfi = get_noised_and_normalized_array(rfi, n_a)
            temp.append(get_intensity(noised_rfi, loc, width))
            
            downsampled_array[i] = noised_rfi
 
        downsampled_array = np.reshape(downsampled_array, (downsampled_array.shape[0], 128, 128, 1))
        intensity_array.append(np.round(np.mean(temp), 1))
        acuracy_list.append(loaded_model.evaluate(downsampled_array,  labels, verbose=0)[1])
 


    with open(f'../files/NBRFI_intensity_sensetivity_for_{name_part}.csv', 'w') as file:
        file.write('Intensity,ACR\n')

        for intensity, acr in zip(intensity_array, acuracy_list):
            file.write(f'{intensity},{acr}\n')

# Visualization of Model Accuracy with Varied Intensity

The accuracy of the models concerning different noise intensities is plotted, providing insights into the models' performance and reliability under different noise conditions and intensities.

In [ ]:
plt.clf()
fig, axs = plt.subplots(1, 2, figsize=(10,5))

# Direct to the correct directory
directory = "../files/"
zoom = (8, 18)

for file in sorted(glob.glob(f'{directory}NBRFI_intensity_sensetivity_for_*.csv')):
    df = pd.read_csv(file)

    # Extracting name_part from filename for the legend
    name_part = os.path.basename(file).split("NBRFI_intensity_sensetivity_for_")[1].split(".csv")[0]

    # Plotting on the first subplot
    axs[0].scatter(df['Intensity'], df['ACR'], label=name_part, s=15)
    axs[0].plot(df['Intensity'], df['ACR'], ls=':')

    # Plotting on the second subplot (zoomed)
    axs[1].scatter(df['Intensity'], df['ACR'], label=name_part, s=15)
    axs[1].plot(df['Intensity'], df['ACR'], ls=':')
    axs[1].set_xlim(8, 18)  # Adjust as necessary for your desired zoom level

# Setting labels and titles
axs[0].set_ylabel('Accuracy of the model')
axs[0].set_xlabel('Intensity')
axs[0].set_title('Full View')
axs[0].grid(True)

# axs[1].set_xscale('log') 
axs[1].set_xlabel('Intensity')
axs[1].set_title(f'Zoomed View ({zoom[0]} < SNR < {zoom[1]})')
axs[1].grid(True)

# Adjusting layout and displaying the legend
plt.tight_layout()
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


# Complex RFI Visualization

Complex RFIs are generated and visualized to examine the representation and characteristics of multi-component RFIs.

In [ ]:
width_list = np.arange(1, 17)
plt.figure(figsize=(10,10))
for i in range(1, 17):
    plt.subplot(4,4,i)
    
    rfi = get_complex_rfi(i)
    noised_rfi = get_noised_and_normalized_array(rfi, 0.01)
    plt.title(f'Num: {i}')
    plt.imshow(noised_rfi, aspect='auto', cmap='gray')
    
plt.tight_layout()    
plt.show()

# Model Evaluation with Varied Number of RFIs

The loaded models are evaluated with different numbers of RFIs to assess their adaptability and accuracy in identifying signals with varied complexities, and the results are documented for further analysis.

In [ ]:
num_spects = 50
labels = np.full(num_spects, 3)
num_list = np.arange(1, 21)
num_list

In [ ]:
for modelname in tqdm(models, desc='Models loop'):
    name_part = os.path.basename(modelname).split('-')[0]
    loaded_model = tf.keras.models.load_model(modelname)
    num_array = []
    acuracy_list = []

    for nums in tqdm(num_list, desc='Nums loop'):
        downsampled_array = np.empty((num_spects, 128, 128))      
        
        for i in trange(num_spects, leave=False):
            rfi = get_complex_rfi(nums)
            noised_rfi = get_noised_and_normalized_array(rfi, 0.01)
            
            downsampled_array[i] = noised_rfi
 
        downsampled_array = np.reshape(downsampled_array, (downsampled_array.shape[0], 128, 128, 1))
        num_array.append(nums)
        acuracy_list.append(loaded_model.evaluate(downsampled_array,  labels, verbose=0)[1])
 
    with open(f'../files/NBRFI_nums_sensetivity_for_{name_part}.csv', 'w') as file:
        file.write('Nums,ACR\n')

        for nums, acr in zip(num_array, acuracy_list):
            file.write(f'{nums},{acr}\n')

# Visualization of Model Accuracy with Varied Number of RFIs

The accuracy of the models concerning different numbers of RFIs is visualized to provide insights into the models' adaptability and reliability under varied signal complexities.

In [ ]:
directory = "../files/"

# Create a single plot
plt.figure(figsize=(10,6))

for file in sorted(glob.glob(f'{directory}NBRFI_nums_sensetivity_for_*.csv')):
    df = pd.read_csv(file)

    # Extracting name_part from filename for the legend
    name_part = os.path.basename(file).split("NBRFI_nums_sensetivity_for_")[1].split(".csv")[0]

    # Plotting on the single plot
    plt.scatter(df['Nums'], df['ACR'], label=name_part, s=15)
    plt.plot(df['Nums'], df['ACR'], ls=':')

# Setting labels, title and legend
plt.ylabel('Accuracy of the model')
plt.xlabel('Number of RFIs')
plt.title('Accuracy by Different number of RFIs for Different Models')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True, ls='--')

# Adjusting layout and displaying the plot
plt.tight_layout()
plt.show()

# Visualization of RFIs with Varied Widths

RFIs with varied pulse widths are generated and visualized, allowing for the analysis of how pulse width impacts the representation of signals in the interference.

In [ ]:
width_list = np.round(np.linspace(1, 30, 25), 2)
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    amp = 1
    loc = 60
    
    nbrfi = NBRFI(128, 128)
    rfi = nbrfi.get_rfi(amp, width_list[i], loc)
    noised_rfi = get_noised_and_normalized_array(rfi, 0.05)
    plt.title(f'W: {width_list[i]}')
    plt.imshow(noised_rfi, aspect='auto', cmap='gray')
    
plt.tight_layout()    
plt.show()

# Model Evaluation with Varied Pulse Widths

The loaded models are evaluated with different pulse widths to assess their adaptability and accuracy in identifying signals with varied widths, and the results are documented for further analysis.

In [ ]:
num_spects = 50
labels = np.full(num_spects, 3)
width_list = np.round(np.linspace(1, 30, 50), 2)
width_list

In [ ]:
for modelname in tqdm(models, desc='Models loop'):
    name_part = os.path.basename(modelname).split('-')[0]
    loaded_model = tf.keras.models.load_model(modelname)
    width_array = []
    acuracy_list = []

    for width in tqdm(width_list, desc='DM loop'):
        downsampled_array = np.empty((num_spects, 128, 128))      
        
        for i in trange(num_spects, leave=False):
            amp = 1
            loc = 60
            
            nbrfi = NBRFI(128, 128)
            rfi = nbrfi.get_rfi(amp, width, loc)
            noised_rfi = get_noised_and_normalized_array(rfi, 0.02)
            
            downsampled_array[i] = noised_rfi
 
        downsampled_array = np.reshape(downsampled_array, (downsampled_array.shape[0], 128, 128, 1))
        width_array.append(width)
        acuracy_list.append(loaded_model.evaluate(downsampled_array,  labels, verbose=0)[1])

    with open(f'../files/NBRFI_width_sensetivity_for_{name_part}.csv', 'w') as file:
        file.write('Width,ACR\n')

        for width, acr in zip(width_array, acuracy_list):
            file.write(f'{width},{acr}\n')

# Visualization of Model Accuracy with Varied Pulse Widths

The accuracy of the models concerning different pulse widths is visualized, providing insights into the models' adaptability and reliability under varied pulse width conditions.

In [ ]:
directory = "../files/"

# Create a single plot
plt.figure(figsize=(10,6))

for file in sorted(glob.glob(f'{directory}NBRFI_width_sensetivity_for_*.csv')):
    df = pd.read_csv(file)

    # Extracting name_part from filename for the legend
    name_part = os.path.basename(file).split("NBRFI_width_sensetivity_for_")[1].split(".csv")[0]

    # Plotting on the single plot
    plt.scatter(df['Width'], df['ACR'], label=name_part, s=15)
    plt.plot(df['Width'], df['ACR'], ls=':')

# Setting labels, title and legend
plt.ylabel('Accuracy of the model')
plt.xlabel('Width')
plt.title('Accuracy by Width for Different Models')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True, ls='--')

# Adjusting layout and displaying the plot
plt.tight_layout()
plt.show()